In [8]:
from surprise import Dataset
from surprise import SVD, model_selection, accuracy

In [2]:
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/JQC/.surprise_data/ml-100k


In [6]:
algo = SVD()
model_selection.cross_validate(algo, data, measures=['rmse', 'mae'], cv=5)

{'test_rmse': array([0.933648  , 0.93376038, 0.94096806, 0.93086491, 0.94000565]),
 'test_mae': array([0.73806831, 0.73363222, 0.74142309, 0.73564343, 0.7417917 ]),
 'fit_time': (4.799232006072998,
  4.582212924957275,
  4.4229607582092285,
  4.390880107879639,
  4.38254714012146),
 'test_time': (0.280637264251709,
  0.19780516624450684,
  0.13890814781188965,
  0.1851179599761963,
  0.1773970127105713)}

In [7]:
model_selection.cross_validate(algo, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9346  0.9356  0.9402  0.9374  0.9324  0.9360  0.0026  
MAE (testset)     0.7370  0.7376  0.7391  0.7401  0.7350  0.7378  0.0018  
Fit time          4.73    4.68    4.37    4.37    4.72    4.58    0.17    
Test time         0.21    0.18    0.18    0.14    0.18    0.18    0.02    


{'test_rmse': array([0.93459957, 0.93557829, 0.94022101, 0.93738563, 0.93237758]),
 'test_mae': array([0.73704928, 0.73763604, 0.73913391, 0.74013213, 0.73496048]),
 'fit_time': (4.733795166015625,
  4.676517963409424,
  4.373437881469727,
  4.368366718292236,
  4.724961996078491),
 'test_time': (0.20508694648742676,
  0.17841887474060059,
  0.1773698329925537,
  0.13898921012878418,
  0.17634987831115723)}

### 第二个demo

In [9]:
trainset, testset = model_selection.train_test_split(data, test_size=0.3, random_state=66)
algo = SVD().fit(trainset)
pred = algo.test(testset)
accuracy.rmse(pred)

RMSE: 0.9397


0.9397103524050693

In [16]:
testset[:10]

[('897', '479', 4.0),
 ('128', '26', 4.0),
 ('476', '72', 4.0),
 ('1', '29', 1.0),
 ('543', '4', 4.0),
 ('14', '473', 5.0),
 ('879', '866', 5.0),
 ('660', '201', 3.0),
 ('477', '255', 5.0),
 ('572', '100', 3.0)]

In [17]:
# uid 用户id iid item的id  r_ui 实际评分  est 预测得分
algo.predict(uid='897', iid='479', r_ui=4, verbose=True)

user: 897        item: 479        r_ui = 4.00   est = 4.71   {'was_impossible': False}


Prediction(uid='897', iid='479', r_ui=4, est=4.713497631070975, details={'was_impossible': False})

### 3.自定义数据集

In [21]:
import os
import pandas as pd
from surprise import Reader, BaselineOnly

In [26]:
file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.data')
reader = Reader(line_format='user item rating timestamp', sep='\t')
data1 = Dataset.load_from_file(file_path, reader)


In [29]:
model_selection.cross_validate(BaselineOnly(), data1, cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9510  0.9403  0.9435  0.9433  0.9433  0.9443  0.0036  
MAE (testset)     0.7568  0.7437  0.7511  0.7468  0.7458  0.7488  0.0047  
Fit time          0.23    0.24    0.33    0.22    0.22    0.25    0.04    
Test time         0.13    0.12    0.12    0.11    0.11    0.11    0.01    


{'test_rmse': array([0.95102154, 0.94026829, 0.94353236, 0.94326756, 0.94333628]),
 'test_mae': array([0.75683065, 0.74367025, 0.75112014, 0.74675584, 0.74577309]),
 'fit_time': (0.23389601707458496,
  0.235673189163208,
  0.32601308822631836,
  0.22138595581054688,
  0.21896100044250488),
 'test_time': (0.12528109550476074,
  0.1153709888458252,
  0.12156295776367188,
  0.10608696937561035,
  0.10554814338684082)}

###  4.加载 DataFrame

In [34]:
from surprise import NormalPredictor

In [31]:
ratings_dict = {'itemID': [1, 1, 1, 2, 2],
                'userID': [9, 32, 2, 45, 'user_foo'],
                'rating': [3, 2, 4, 3, 1]}
df = pd.DataFrame(ratings_dict)
df

,itemID,userID,rating
0,1,9,3
1,1,32,2
2,1,2,4
3,2,45,3
4,2,user_foo,1


In [32]:
reader = Reader(rating_scale=(1, 5))
data2 = Dataset.load_from_df(df, reader)

In [35]:
model_selection.cross_validate(NormalPredictor(), data2, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1858  0.5304  2.0644  1.9750  0.0783  0.9668  0.8730  
MAE (testset)     0.1858  0.5304  2.0644  1.9750  0.0783  0.9668  0.8730  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.18576497, 0.53044769, 2.06436831, 1.97499036, 0.07834425]),
 'test_mae': array([0.18576497, 0.53044769, 2.06436831, 1.97499036, 0.07834425]),
 'fit_time': (0.00013709068298339844,
  9.012222290039062e-05,
  5.0067901611328125e-05,
  4.315376281738281e-05,
  4.100799560546875e-05),
 'test_time': (5.888938903808594e-05,
  4.1961669921875e-05,
  3.1948089599609375e-05,
  2.574920654296875e-05,
  2.6226043701171875e-05)}

### 5.使用K折

In [37]:
data = Dataset.load_builtin('ml-100k')

In [39]:
kfold = model_selection.KFold(n_splits=5, random_state=66)
algo = SVD()
for trainset, testset in kfold.split(data):
    algo.fit(trainset)
    pred = algo.test(testset)
    accuracy.rmse(pred, verbose=True)

RMSE: 0.9321
RMSE: 0.9441
RMSE: 0.9418
RMSE: 0.9306
RMSE: 0.9313


In [40]:
model_selection.cross_validate(algo, data, cv=kfold, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9282  0.9468  0.9414  0.9303  0.9381  0.9370  0.0069  
MAE (testset)     0.7315  0.7456  0.7409  0.7333  0.7394  0.7381  0.0051  
Fit time          4.52    4.41    5.01    4.57    4.37    4.58    0.23    
Test time         0.15    0.36    0.18    0.14    0.14    0.19    0.09    


{'test_rmse': array([0.92821623, 0.94684703, 0.9414493 , 0.93025975, 0.93811552]),
 'test_mae': array([0.73151069, 0.7455829 , 0.74094127, 0.73332344, 0.73936745]),
 'fit_time': (4.523874998092651,
  4.409788131713867,
  5.00588321685791,
  4.568657875061035,
  4.372728109359741),
 'test_time': (0.14788484573364258,
  0.3624849319458008,
  0.18331003189086914,
  0.13824892044067383,
  0.13678789138793945)}

### 6.加载多个文件数组

In [47]:
file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/')
reader = Reader('ml-100k')

train_file = file_path + 'u%d.base'
test_file = file_path + 'u%d.test'
folds_files = [(train_file % i, test_file % i) for i in range(1, 6)]


[('/Users/JQC/.surprise_data/ml-100k/ml-100k/u1.base',
  '/Users/JQC/.surprise_data/ml-100k/ml-100k/u1.test'),
 ('/Users/JQC/.surprise_data/ml-100k/ml-100k/u2.base',
  '/Users/JQC/.surprise_data/ml-100k/ml-100k/u2.test'),
 ('/Users/JQC/.surprise_data/ml-100k/ml-100k/u3.base',
  '/Users/JQC/.surprise_data/ml-100k/ml-100k/u3.test'),
 ('/Users/JQC/.surprise_data/ml-100k/ml-100k/u4.base',
  '/Users/JQC/.surprise_data/ml-100k/ml-100k/u4.test'),
 ('/Users/JQC/.surprise_data/ml-100k/ml-100k/u5.base',
  '/Users/JQC/.surprise_data/ml-100k/ml-100k/u5.test')]

In [48]:
# PredefinedKFold 适用于load_from_folds加载多文件
data3 = Dataset.load_from_folds(folds_files, reader)
pkf = model_selection.PredefinedKFold()
algo = SVD()
for trainset, testset in pkf.split(data3):
    algo.fit(trainset)
    pred = algo.test(testset)
    accuracy.rmse(pred, verbose=True)

RMSE: 0.9512
RMSE: 0.9386
RMSE: 0.9323
RMSE: 0.9317
RMSE: 0.9335


### 7.网格搜索寻找最佳的参数

In [49]:
data = Dataset.load_builtin('ml-100k')

In [51]:
"""
   n_factors=100,
    n_epochs=20,
    biased=True,
    init_mean=0,
    init_std_dev=0.1,
    lr_all=0.005,
    reg_all=0.02,
"""
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = model_selection.GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=6)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


0.9634621999832996
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [52]:
"""
KFold基础k折交叉验证

RepeatedKFold 多次k折交叉验证.

ShuffleSplit乱序训练集和数据集下的基础交叉验证

LeaveOneOut在测试集上每个用户只取一个评分做交叉验证

PredefinedKFold：数据集是通过方法 load_from_folds 加载进来的交叉验证方法.
"""



'\nKFold基础k折交叉验证\n\nRepeatedKFold\xa0多次k折交叉验证.\n\nShuffleSplit乱序训练集和数据集下的基础交叉验证\n\nLeaveOneOut在测试集上每个用户只取一个评分做交叉验证\n\nPredefinedKFold：数据集是通过方法\xa0load_from_folds\xa0加载进来的交叉验证方法.\n'